In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Numeric, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from faker import Faker
from datetime import datetime, timedelta
import random
import pandas as pd

In [2]:
# Crear la conexión a la base de datos en memoria
engine = create_engine('sqlite:///:memory:')

In [3]:
# Crea una clase base para la definición de tablas
Base = declarative_base()

In [4]:
# Definir la tabla tb_users
class User(Base):
    __tablename__ = 'tb_users'
    
    user_id = Column(Integer, primary_key=True)
    created_dt = Column(Date)
    name = Column(String(60))
    email = Column(String(100))
    phone = Column(Integer)
    birthdate = Column(Date)
    gender = Column(String(6))
    street_address = Column(String(100))
    cp = Column(Integer)
    country = Column(String(56))


In [5]:
# Definir la tabla tb_loan
class Loan(Base):
    __tablename__ = 'tb_loan'
    
    loan_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('tb_users.user_id'))
    contract_start_date = Column(Date)
    contract_end_date = Column(Date)
    loan_amount = Column(Numeric(6, 2))
    interest_rate = Column(Numeric(2, 2))
    payment_frequency = Column(String(5))
    loan_term = Column(Integer)

In [6]:
# Definir la tabla tb_payment
class Payment(Base):
    __tablename__ = 'tb_payment'
    
    payment_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('tb_users.user_id'))
    loan_id = Column(Integer, ForeignKey('tb_loan.loan_id'))
    limit_payment_dt = Column(Date)
    payment_date = Column(Date)
    payment_method = Column(String(10))
    hash_card = Column(String(10))
    issue_bank = Column(String(20))
    payment_amount = Column(Numeric(9, 2))

In [7]:
# Crear las tablas en la base de datos
Base.metadata.create_all(engine)

In [8]:
# Crear una sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
# Crear una instancia de Faker
fake = Faker()

In [12]:
# Generar datos aleatorios para las tablas
for i in range(1000):  # El número en rango será el número de rows que tengas
# Generar datos para la tabla tb_users
    user = User(
        created_dt=fake.date_this_decade(),
        name=fake.name(),
        email=fake.email(),
        phone=fake.random_int(1000000000, 9999999999),
        birthdate=fake.date_of_birth(),
        gender=random.choice(['Male', 'Female']),
        street_address=fake.street_address(),
        cp=random.choice([44150,44214,44219,44248,44266]),
        country=fake.country()
    )
    session.add(user)

In [13]:
# Generar datos aleatorios para las tablas
for i in range(1000):  # El número en rango será el número de rows que tengas 
# Generar datos para la tabla tb_loan
    loan = Loan(
        user_id=random.randint(1, 10),  # Ajusta esto según la cantidad de registros generados
        contract_start_date=fake.date_this_year(),
        contract_end_date=fake.date_this_year() + timedelta(days=random.randint(30, 365)),
        loan_amount=round(random.uniform(1000, 50000)),
        interest_rate=random.uniform(0.01, 0.2),
        payment_frequency=random.choice(['monthly']),
        loan_term=random.randint(12, 72)
    )
    session.add(loan)

In [14]:
# Generar datos aleatorios para las tablas
for i in range(1000):  # El número en rango será el número de rows que tengas
    
# Generar datos para la tabla tb_payment
    payment = Payment(
        user_id=random.randint(1, 10),  # Ajusta esto según la cantidad de registros generados
        loan_id=random.randint(1, 10),  # Ajusta esto según la cantidad de registros generados
        limit_payment_dt=fake.date_this_year(),
        payment_date=fake.date_this_year(),
        payment_method=random.choice(['Debit Card', 'Cash']),
        hash_card=fake.credit_card_number(),
        issue_bank=fake.random_element(['Santander', 'BBVA', 'Azteka', 'Banamex']),
        payment_amount=round(random.uniform(50, 5000)),
    )
    session.add(payment)

In [15]:
# Se hace commit para guardar los datos en la base de datos
session.commit()

In [16]:
users_df = pd.read_sql(session.query(User).statement, session.bind)
loans_df = pd.read_sql(session.query(Loan).statement, session.bind)
payments_df = pd.read_sql(session.query(Payment).statement, session.bind)

C:\ProgramData\anaconda3\lib\site-packages\pandas\io\sql.py:1405: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return self.connectable.execution_options().execute(*args, **kwargs)
C:\ProgramData\anaconda3\lib\site-packages\pandas\io\sql.py:1405: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return self.connectable.execution_options().execute(*args, **kwargs)


In [17]:
# Convertir columnas Numeric a strings para evitar problemas de precisión al exportar a CSV
loans_df['loan_amount'] = loans_df['loan_amount'].astype(str)
loans_df['interest_rate'] = loans_df['interest_rate'].astype(str)
loans_df['loan_term'] = loans_df['loan_term'].astype(str)

In [18]:
users_df.to_csv('users.csv', index=False)
loans_df.to_csv('loans.csv', index=False)
payments_df.to_csv('payments.csv', index=False)